In [9]:
try:
    import pyserial
except:
    import pip
    pip.main(['install', 'pyserial', 'tweepy', 'textblob', 'nltk'])

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
data = pd.read_csv("tweets1.csv")

In [11]:
list(data.columns.values)

['Brexit_Tweet', 'Opinion']

In [12]:
data.head()

,Brexit_Tweet,Opinion
0,RT @Holbornlolz: Corbyn Brexit speech summary:...,Positive
1,RT @LeaveEUOfficial: WATCH | Nick Ferrari has ...,Positive
2,@AdamAfriyie @PlanB_earth @friends_earth @NoR3...,Neutral
3,RT @AngryScotland: The most prominent of the p...,Positive
4,RT @OFOCBrexit: Watch what happened on #Victor...,Neutral


In [13]:
data.dropna(how='any')

,Brexit_Tweet,Opinion
0,RT @Holbornlolz: Corbyn Brexit speech summary:...,Positive
1,RT @LeaveEUOfficial: WATCH | Nick Ferrari has ...,Positive
2,@AdamAfriyie @PlanB_earth @friends_earth @NoR3...,Neutral
3,RT @AngryScotland: The most prominent of the p...,Positive
4,RT @OFOCBrexit: Watch what happened on #Victor...,Neutral
5,And yet still their headline vitriol is read o...,Neutral
6,RT @WestmonsterUK: ANGER: Anti-No Deal MP faci...,Negative
7,"Well said, I don't recall those Politicians ig...",Neutral
8,RT @mrjamesob: The fact that Jaguar Land Rover...,Neutral
9,RT @davidschneider: 1 in 5 children now live w...,Positive


In [14]:
import nltk


stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

def normalizer(tweet):
    only_letters = re.sub("[^a-zA-Z]", " ",tweet) 
    tokens = nltk.word_tokenize(only_letters)[2:]
    lower_case = [l.lower() for l in tokens]
    filtered_result = list(filter(lambda l: l not in stop_words, lower_case))
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
    return lemmas

In [16]:
normalizer("Positive, Negative, Neutral")

['neutral']

In [17]:
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,2))

In [20]:
vectorized_data = count_vectorizer.fit_transform(data.Brexit_Tweet)
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))

In [21]:
def sentiment2target(sentiment):
    return {
        'Negative': 0,
        'Neutral': 1,
        'Positive' : 2
    }[sentiment]
targets = data.Opinion.apply(sentiment2target)

In [22]:
from sklearn.model_selection import train_test_split
data_train, data_test, targets_train, targets_test = train_test_split(indexed_data, targets, test_size=0.3, random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

In [23]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf_output = clf.fit(data_train, targets_train)

In [24]:
clf.score(data_test, targets_test)

0.89311594202898548